In [25]:
# extract expression values for each transcript in chess for each tissue

# format of the input file
# ALL_03716214	SRR1466260:0.652967
# ALL_02055884	SRR1445170:0.560959	SRR661373:1.153505	SRR818850:0.479766
# ALL_10044090	SRR1069734:2.544744
# ALL_18714109	SRR1101373:0.678579
# ALL_17680799	SRR1081116:1.300604	SRR1356643:1.430289
# ALL_25719799	SRR613963:1.111076
# ALL_13221907	SRR1343053:3.592212
# ALL_22293686	SRR820733:0.413197
# ALL_08997145	SRR1079046:1.633669	SRR1090070:2.432191	SRR1351390:2.003746	SRR1365013:1.225308	SRR1442592:0.674933	SRR1456693:0.899244	SRR659637:0.5877
# ALL_04709978	SRR1436858:2.427112	SRR615443:2.889572	SRR819771:0.835807

# we can extract chess id to all id mapping from the chess gtf file

# then we also get the list of sample ids to bodysites and tissues as follows

In [26]:
import os
import sys
import time
import glob
import shutil
import argparse
import subprocess
import numpy as np
import pandas as pd

In [27]:
non_zero_tpms_fname = "/ccb/salz8-1/avaraby/chess3.1/data/ALL.expression.nonzero.tpms"
chess_gtf_fname = "/home/avaraby1/genomes/human/hg38/annotations/chess3.1.3.GRCh38.gtf"
info_fname = "/ccb/salz8-1/avaraby/GTEx_metadata/GTEx_salmples_info.tab"

In [28]:
# load info
info = pd.read_csv(info_fname,sep="\t")
info

,LibraryLayout,MBases,MBytes,Run,body_site,histological_type,is_tumor,sex
0,PAIRED,5876,2417,SRR1098761,Cells - Cultured fibroblasts,Skin,No,male
1,PAIRED,4833,2206,SRR1098785,Lung,Lung,No,male
2,PAIRED,5072,2326,SRR1098807,Ovary,Ovary,No,female
3,PAIRED,4449,1864,SRR1098831,Esophagus - Muscularis,Esophagus,No,female
4,PAIRED,5737,2709,SRR1098855,Stomach,Stomach,No,female
...,...,...,...,...,...,...,...,...
24601,PAIRED,5739,2444,SRR1094168,Cells - Transformed fibroblasts,Skin,No,male
24602,PAIRED,4944,2150,SRR1097076,Esophagus - Muscularis,Esophagus,No,male
24603,PAIRED,4403,1887,SRR1097905,Prostate,Prostate,No,male
24604,PAIRED,5394,2311,SRR1100564,Artery - Aorta,Blood Vessel,No,male


In [29]:
# group samples by bodysite instead of tissue

bodysite_to_run = dict()
run_to_bodysite = dict()
tissue_to_bodysite = dict()
bodysite_to_tissue = dict()

for idx,row in info.iterrows():
    bs = row["body_site"].replace(" - ","_").replace(" ",".").replace("(","").replace(")","")
    hist_type = row["histological_type"].replace(" ","_")
    
    bodysite_to_run.setdefault(bs,[])
    bodysite_to_run[bs].append(row["Run"])
    
    run_to_bodysite.setdefault(row["Run"],None)
    assert run_to_bodysite[row["Run"]] is None or run_to_bodysite[row["Run"]]==bs
    run_to_bodysite[row["Run"]]=bs
    
    tissue_to_bodysite.setdefault(hist_type,[])
    tissue_to_bodysite[hist_type].append(bs)
    
    bodysite_to_tissue.setdefault(bs,None)
    assert bodysite_to_tissue[bs] is None or bodysite_to_tissue[bs]==hist_type
    bodysite_to_tissue[bs]=hist_type
    
print("number of bodysites: "+str(len(bodysite_to_run)))

for bs in bodysite_to_run:
    print(bs,len(bodysite_to_run[bs]))

number of bodysites: 55
Cells_Cultured.fibroblasts 666
Lung 891
Ovary 254
Esophagus_Muscularis 699
Stomach 487
Muscle_Skeletal 1105
Skin_Sun.Exposed.Lower.leg 928
Pancreas 469
Cells_EBV-transformed.lymphocytes 285
Esophagus_Mucosa 788
Brain_Putamen.basal.ganglia 243
Adipose_Subcutaneous 871
Brain_Nucleus.accumbens.basal.ganglia 290
Artery_Tibial 827
Whole.Blood 2797
Colon_Transverse 500
Adrenal.Gland 368
Prostate 282
Esophagus_Gastroesophageal.Junction 440
Brain_Frontal.Cortex.BA9 267
Thyroid 858
Adipose_Visceral.Omentum 598
Artery_Aorta 568
Uterus 216
Small.Intestine_Terminal.Ileum 249
Bladder 24
Breast_Mammary.Tissue 530
Testis 493
Minor.Salivary.Gland 175
Skin_Not.Sun.Exposed.Suprapubic 683
Brain_Cerebellar.Hemisphere 276
Liver 332
Vagina 227
Brain_Anterior.cingulate.cortex.BA24 236
Pituitary 324
Fallopian.Tube 14
Spleen 290
Colon_Sigmoid 429
Cells_Leukemia.cell.line.CML 248
Artery_Coronary 325
Heart_Atrial.Appendage 531
Kidney_Cortex 88
Brain_Substantia.nigra 171
Brain_Cortex 311
B

In [30]:
# get all_id mapping from chess gtf

all2chs = dict()

with open(chess_gtf_fname,"r") as f:
    for line in f:
        if line.startswith("#"):
            continue
        lcs = line.strip().split("\t")
        if lcs[0] != "chr21":
            continue
        if lcs[2] == "transcript":
            aid = None
            tid = None
            attrs = lcs[8].strip().split(";")
            for attr in attrs:
                attr = attr.strip()
                k,v = attr.split(" \"",1)
                v = v.strip('"')
                if k == "assembly_id":
                    aid = v
                if k == "transcript_id":
                    tid = v
            if aid is not None and tid is not None:
                all2chs[aid] = tid


In [31]:
# iterate over the expression file, check if all id is in the all2chs dict
# if exists - group expressions by bodysite
# if not exists - skip

# load all2chs

chs_grouped_expressions = dict()

with open(non_zero_tpms_fname,"r") as f:
    for line in f:
        lcs = line.strip().split("\t")
        all_id = lcs[0]
        if all_id not in all2chs:
            continue
        
        chs_id = all2chs[all_id]
        if chs_id not in chs_grouped_expressions:
            chs_grouped_expressions[chs_id] = dict()

        # iterate over remaining columns - each columns is run:expression value
        # run can be looked up in run_to_bodysite

        for i in range(1,len(lcs)):
            run,expr = lcs[i].split(":")
            try:
                expr = float(expr)
            except:
                continue
            if run not in run_to_bodysite:
                continue
            bodysite = run_to_bodysite[run]
            if bodysite not in chs_grouped_expressions[chs_id]:
                chs_grouped_expressions[chs_id][bodysite] = []
            chs_grouped_expressions[chs_id][bodysite].append(expr)


In [33]:
# write output file in the following format:
# chess_id\tbodysite1:percentile_data;bodysite2:percentile_data

def get_percentiles(data,precision=2):
    result = dict()
    for p in [25,50,75]:
        result[p] = np.round(np.percentile(data, p),precision)
    result["min"] = np.round(np.min(data),precision)
    result["max"] = np.round(np.max(data),precision)
    result["mean"] = np.round(np.mean(data),precision)
    result["std"] = np.round(np.std(data),precision)
    result["count"] = len(data)
    return result

with open("./chess_expression.chr21.tsv","w") as outFP:
    for chs_id in chs_grouped_expressions:
        outFP.write(chs_id+"\t")
        for bodysite in chs_grouped_expressions[chs_id]:
            percentile_data = get_percentiles(chs_grouped_expressions[chs_id][bodysite])
            percentile_data_str = ",".join([f"{k}:{v}" for k,v in percentile_data.items()])
            outFP.write(bodysite+":"+percentile_data_str+";")
        outFP.write("\n")